In [1]:
import pandas as pd
import pyarrow

anime = pd.read_csv("anime.csv")
rating = pd.read_csv("rating.csv")





In [2]:
rating.shape

(7813737, 3)

In [3]:
anime.isnull().sum()

Unnamed: 0      0
anime_id        0
name            0
genre          62
type           25
episodes        0
rating        230
members         0
dtype: int64

In [4]:
rating.isnull().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [5]:
anime.duplicated().sum()

0

In [6]:
rating.duplicated().sum()

1

In [7]:
merged = rating.merge(anime,on='anime_id')

In [8]:
merged

,user_id,anime_id,rating_x,Unnamed: 0,name,genre,type,episodes,rating_y,members
0,1,20,-1,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,3,20,8,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
2,5,20,6,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
3,6,20,-1,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
4,10,20,-1,841,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
...,...,...,...,...,...,...,...,...,...,...
7813722,65682,30450,8,8493,Dr. Slump: Hoyoyo! Arale no Himitsu Dai Koukai...,"Comedy, Sci-Fi, Shounen",Special,1,6.17,248
7813723,69497,33484,10,10256,Shiroi Zou,"Action, Historical, Kids",Movie,1,4.71,45
7813724,70463,29481,-1,9097,Kakinoki Mokkii,"Fantasy, Kids",Special,1,4.33,61
7813725,72404,34412,-1,8777,Hashiri Hajimeta bakari no Kimi ni,Music,Music,1,6.76,239


In [9]:
numr = merged.groupby('anime_id').count()['rating_x'].reset_index()
numr.rename(columns={'rating_x':'num_ratings'},inplace=True)
numr

,anime_id,num_ratings
0,1,15509
1,5,6927
2,6,11077
3,7,2629
4,8,413
...,...,...
11192,34367,5
11193,34412,1
11194,34475,4
11195,34476,1


In [10]:
numr.head(5000)

,anime_id,num_ratings
0,1,15509
1,5,6927
2,6,11077
3,7,2629
4,8,413
...,...,...
4995,7430,73
4996,7435,13
4997,7436,14
4998,7445,9


In [11]:
x = merged.groupby('user_id').count()['rating_x'] > 150

In [12]:
cultured = x[x].index

filtered = merged[merged['user_id'].isin(cultured)]

y = filtered.groupby('anime_id').count()['rating_x']>=150
eanime = y[y].index

final = filtered[filtered['anime_id'].isin(eanime)]

lfinal = final.pivot_table(index='anime_id',columns='user_id',values='rating_x')

lfinal.fillna(0,inplace=True)

In [13]:
lfinal



user_id,1,5,7,13,17,21,38,39,43,46,...,73478,73485,73491,73495,73499,73500,73502,73503,73507,73515
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,-1.0,0.0,9.0,0.0,0.0,10.0,10.0,...,10.0,10.0,0.0,10.0,9.0,0.0,0.0,9.0,9.0,10.0
5,0.0,0.0,0.0,-1.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-1.0,8.0,0.0,0.0,0.0,7.0,8.0,10.0
6,0.0,8.0,0.0,-1.0,7.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,-1.0,9.0,0.0,0.0,9.0,9.0,10.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,7.0,0.0,0.0,9.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33569,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
